In [1]:
import agent_code.vkl.models as models
from agent_code.vkl.data import MoveDataset
from torch.utils.data import DataLoader
from torch import load
from pytorch_lightning.callbacks import TQDMProgressBar
import pytorch_lightning as L

# Parameters

In [2]:
radius = 1

# Data

In [3]:
packed = load("agent_code/watcher/output/moves_0.pt")
dataset = MoveDataset(packed, radius=radius)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

/tmp/ipykernel_1366300/2149004423.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  packed = load("agent_code/watcher/output/moves_0.pt")


# Model definition

In [4]:
raw_model = models.BasicModel(radius)
model = models.LitBasicModel(raw_model)

final conv layer size: 1


# Training

In [5]:
trainer = L.Trainer(accelerator="gpu", max_epochs=100, callbacks=[TQDMProgressBar(refresh_rate=1)])
trainer.fit(model=model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | BasicModel | 70.4 K | train
---------------------------------------------
70.4 K    Trainable params
0         Non-trainable params
70.4 K    Total params
0.282     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode
/nix/store/31sjmwkb2rypzm6b759l7ncq3k2kzzvr-python3-3.12.5-env/lib/python3.12/site-packages/pytorch_l

Training: |                                                                                                   …

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined